In [1]:
from frontend.database import session
from scripts import generate_fc_layer_circuit, benchmark_utils
import numpy as np

In [2]:
weight_size = 2000
weight = np.asarray([int(np.random.random() > 0.5) for i in range(weight_size)])

In [3]:
circuit_file=generate_fc_layer_circuit.generate(weight.size, weight)
print(circuit_file)

/Users/asanyal/research/SHEEP//benchmark_inputs/mid_level/circuits/circuit-fc_layer-2000.sheep


In [4]:
inp_var = benchmark_utils.get_inputs(circuit_file)
out_var = benchmark_utils.get_outputs(circuit_file)
inp_dict = {}
for inps in inp_var:
    inp_dict[inps] = int(np.random.random() > 0.5)

In [5]:
inputs_file = benchmark_utils.write_inputs_file(inp_dict)

In [6]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"bool","TFHE")

In [34]:
assert np.alltrue([int(weight[i]==inp_dict['FCINPUT_'+str(i)]) == int(results['Outputs']['XNOROUTPUT_'+str(i)]) for i in range(weight_size)])

In [35]:
print(results['Processing times (s)']['circuit_evaluation'])

0.00484


In [11]:
from scripts.rocks import circuit
from scripts.particles import variables
from scripts.reusable_modules import nb_adder

In [12]:
x = variables('x')
y = variables('y')
cin = variables('cin')
s = variables('sum')
c = variables('carry')
b2_adder = nb_adder('2b_adder', [x, y, cin], [s, c], 2)
c_obj = circuit('2 Bit Adder', b2_adder)
c_obj.write_file(filename='./test.sheep')
inp_dict = {'x':0, 'y':1, 'cin':1}
inputs_file = benchmark_utils.write_inputs_file(inp_dict)
results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")

In [13]:
results

{'Cleartext check': {'is_correct': True},
 'Object sizes (bytes)': {u'ciphertext': u'24',
  u'privateKey': u'48',
  u'publicKey': u'48'},
 'Outputs': {u'carry': u'1', u'sum': u'0'},
 'Parameter values': {u'MinimumLambda': u'110'},
 'Processing times (s)': {u'circuit_evaluation': '0.12925',
  u'decryption': '0.0',
  u'encryption': '8e-05',
  u'setup': '0.41446'},
 'context': 'TFHE',
 'input_bitwidth': 1,
 'input_signed': False}